#Imports

In [1]:
# Drive Mounting (for Google Colab only). If not using Colab, comment the below two lines.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon

In [3]:
root="/content/drive/MyDrive/Folder_Name/"               #path of the root directory

#Data Loading

In [4]:
publaydata = np.load(root+"publaynet.npy")
Transformer_res=np.load(root+"trans.npy")
VAE_res = np.load(root+"VAE_res.npy")
GAN_res = np.load(root+"GAN_res.npy")

Reshaping and arranging data in an optimal format. Preferred format is [c x y w h]

* c is the class of the box.
* x and y are the corrdinates for the top left corner of the box.
* w and h are the width and height respectively. 
---



In [5]:
g_data = GAN_res.reshape((1024, 9, 9))[:1000]
g_data = g_data[...,0:5]
g_data = g_data[...,[4,0,1,2,3]]
g_data[...,1] = g_data[...,1] - g_data[...,3]/2
g_data[...,2] = g_data[...,2] - g_data[...,4]/2

g_data[0]

array([[0.37773186, 0.06993452, 0.19879478, 0.93849486, 0.37714368],
       [0.63268024, 0.08424908, 0.09495369, 0.7752126 , 0.770834  ],
       [0.31078017, 0.22171676, 0.65355074, 0.6689019 , 0.08199155],
       [0.9312403 , 0.20145085, 0.0982542 , 0.66996616, 0.05629438],
       [0.6019517 , 0.11892939, 0.76413304, 0.7681484 , 0.15887733],
       [0.31345773, 0.22002116, 0.64820474, 0.67002124, 0.08382312],
       [0.31812534, 0.22010046, 0.6488024 , 0.6710409 , 0.08644559],
       [0.36010844, 0.07238191, 0.18851566, 0.9383161 , 0.40076703],
       [0.93621445, 0.20135537, 0.09839028, 0.66991025, 0.05546013]],
      dtype=float32)

#Losses
Overall three losses are discussed for the comparison:
* Overlapping
* Alignment
* IoU

The expressions for the calculations can be found in the Readme shared.

In [6]:
def overlapping_loss(result):
    losses=np.zeros(len(result))
    idx=0
    for i in result:
        over=0
        for j in range(len(i)):
            A=float(i[j][3]*i[j][4])
            if A==0:
                continue
            for k in range(len(i)):
                if j==k:
                    continue
                x1=i[j][1]
                x2=i[j][1]+i[j][3]
                y1=i[j][2]
                y2=i[j][2]+i[j][4]
                x3=i[k][1]
                x4=i[k][1]+i[k][3]
                y3=i[k][2]
                y4=i[k][2]+i[k][4]
                x_over=max(min(x2,x4)-max(x1,x3),0)
                y_over=max(min(y2,y4)-max(y1,y3),0)
                over+=x_over*y_over/A
        losses[idx]=over
        idx+=1
    return np.mean(losses)*100

In [7]:
def alignment_loss(result):
    xl =result[...,1]           
    yl = result[...,2]
    
    xr = xl+result[...,3]
    yr = yl + result[...,4]

    xc = (xl + xr)/2
    yc = (yl + yr)/2

    ele = [xl , yl , xc, yc, xr, yr]
    ele1 = []
    epsilon = 0
    for element in ele:
        min_xl = np.ones(shape = element.shape)
        for i in range(len(element)):
            for j in range(len(element[i])):
                for k in range(len(element[i])): 
                    if j != k :
                        min_xl[i][j] = min(min_xl[i][j],abs(element[i][j]-element[i][k]))        
        min_xl = -np.log(1.0-min_xl + epsilon)
        ele1.append(min_xl)
    ele1 = np.min(np.array(ele1), axis = 0)
    ele1 = np.mean(np.sum(ele1 , axis  = 1))
    return ele1*100

In [8]:
def calculate_iou(result):
    losses=np.zeros(len(result))
    idx=0
    for i in result:
        iou=0
        for j in range(len(i)):
            for k in range(j+1,len(i)):
                x1=i[j][1]
                x2=i[j][1]+i[j][3]
                y1=i[j][2]
                y2=i[j][2]+i[j][4]
                x3=i[k][1]
                x4=i[k][1]+i[k][3]
                y3=i[k][2]
                y4=i[k][2]+i[k][4]

                box_1 = [[x1, y1], [x2, y1], [x2, y2], [x1, y2]]
                box_2 = [[x3, y3], [x4, y3], [x4, y4], [x3, y4]]

                poly_1 = Polygon(box_1)
                poly_2 = Polygon(box_2)

                if poly_1.union(poly_2).area!=0:
                    iou += poly_1.intersection(poly_2).area / poly_1.union(poly_2).area
        losses[idx]=iou
        idx+=1
    return np.mean(losses)*100

#Data Comparison
Calculation of metrics for original data of 1000 documents and for all models for 1000 documents (from unseen data).

In [9]:
overlist = [overlapping_loss(publaydata[0:1000]), overlapping_loss(g_data[0:1000]), overlapping_loss(VAE_res[0:1000]),overlapping_loss(Transformer_res)]
ioulist = [calculate_iou(publaydata[0:1000]), calculate_iou(g_data[0:1000]), calculate_iou(VAE_res[0:1000]),calculate_iou(Transformer_res)]
alignlist = [alignment_loss(publaydata[0:1000]), alignment_loss(g_data[0:1000]), alignment_loss(VAE_res[0:1000]),alignment_loss(Transformer_res)]

In [10]:
#normalizing the lists w.r.t the original data
overlist/=overlist[0]
ioulist/=ioulist[0]
alignlist/=alignlist[0]

rows = ["Original Data", "LayoutGAN", "LayoutVAE","Layout Transformer"]
df=pd.DataFrame(index=rows)
df["Overlap"]=overlist
df["IOU"]=ioulist
df["Alignment"]=alignlist

In [11]:
print("Comparison for Publaynet Dataset")
display(df)

Comparison for Publaynet Dataset


,Overlap,IOU,Alignment
Original Data,1.000000,1.000000,1.000000
LayoutGAN,1172.005234,2745.437529,1.164882
LayoutVAE,119.320127,185.864381,3.493406
Layout Transformer,1.090315,1.422297,0.739862
